In [1]:
import pandas as pd
import seaborn as sb
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import numpy as np
import plotly.figure_factory as ff
import dataframe_image as dfi
import pylatex as pl
from pylatex import Command, Document, Section, Subsection
from pylatex.utils import NoEscape, italic
from mdutils.mdutils import *
from mdutils import *
import os

In [2]:
def create_euc_page(wheel):

    wheel_trips=[]
    print('analysis/'+wheel)
    wheel_trips=os.listdir('analysis/'+wheel)
        
    
    portraits = os.listdir('imgs/wheels/portrait')
    portrait = [item for item in portraits if item.startswith(wheel)][0]
    
    wheel_specs = pd.read_csv("specs/"+wheel+".csv",sep='|')

    brand=wheel_specs[wheel_specs['Field']=="Brand"].values
    brand=brand[0][1].lstrip()
    
    year=wheel_specs[wheel_specs['Field']=="Year"].values
    year=year[0][1].lstrip()
    
    wheel_specs_tab=wheel_specs.values.copy()
    for i in range(0,len(wheel_specs_tab)):
       wheel_specs_tab[i]=["**"+wheel_specs_tab[i][0]+"**",wheel_specs_tab[i][1]]
    wheel_specs_tab = ['**Field**','**Data**']+[item for row in wheel_specs_tab for item in row]
    
    battery_cap=wheel_specs[wheel_specs["Field"]=="Battery"]
    battery_cap=battery_cap.values
    battery_cap=battery_cap[0][1].split("Wh")[0]
    battery_cap=int(battery_cap)/1000

    stats_dir="stats/"+wheel
    analysis_dir='analysis/'+wheel
    
    env_stats=pd.read_csv(stats_dir+'/env.csv')
    env=env_stats[['trip_distance_km','rider_weight_kg','tire_pressure_bars','outdoor_temperature_c','altitude_difference']]
    env=env.describe()
    
    env_stats['count']=1
    trips=env_stats[['trip_type','count']].groupby('trip_type').count()
    apps=env_stats[['data_origin','count']].groupby('data_origin').count()
    
    env=env.style.background_gradient()

    conso_stats=pd.read_csv(stats_dir+'/conso.csv')
    conso=conso_stats.describe()
    avg_conso = conso[['average_wh/km']].values[1][0]
    avg_conso=round(avg_conso,2)
    avg_sag = conso[['voltage_sag_coeff']].values[1][0]
    avg_sag = """<span style="color:red;font-weight: bold;">"""+str(round(avg_sag,2))+"""</span>""" if avg_sag < -0.6 else """<span style="color:green;font-weight: bold;">"""+str(round(avg_sag,2))+"""</span>"""
    estimated_range = int(battery_cap/avg_conso*1000)

    conso=conso.style.background_gradient()

    sum_stats=pd.read_csv(stats_dir+'/summary.csv')
    sum_stats=sum_stats.drop(['trip_id'],axis=1)
    
    dfs={}
    
    cols = sum_stats.columns[1:-1]
    for c in cols:
        df=sum_stats[['stat',c]].groupby('stat').describe()
        dfs.update({c:df})
        
    avg_speed=dfs.get("speed")
    avg_speed=avg_speed["speed","mean"]
    avg_speed=round(avg_speed['mean'],2)
    efficiency = avg_conso/avg_speed

    avg_conso = """<span style="color:red;font-weight: bold;">"""+str(avg_conso)+"""</span>""" if efficiency > 1.0 else """<span style="color:green;font-weight: bold;">"""+str(avg_conso)+"""</span>""" if efficiency < 0.75 else """<span style="color:orange;font-weight: bold;">"""+str(avg_conso)+"""</span>"""
    
    computed_specs_tab =['','',"**Average energy consumption (based on data)**","**"+str(avg_conso)+" Wh/km**",
                         '**Estimated range (based on data)**',"**"+str(estimated_range)+" km**",
                         '**Average voltage sag coeffcient**',avg_sag+" in [-1.0;0.0]"]
    
    mdFile = MdUtils(file_name="analysis/"+wheel,title='')
    # Additional Markdown syntax...
    
    mdFile.new_header(level=1, title=brand+" " + wheel.replace("_"," ").upper())
    mdFile.write('**RELEASE YEAR :**  '+year)
    
    mdFile.new_header(level=2, title="Portrait")
    mdFile.new_paragraph(Html.image(path="../imgs/wheels/portrait/"+portrait, size='200'))
    mdFile.new_line()
    
    mdFile.new_header(level=2, title="Specifications")
    mdFile.new_line()
    mdFile.new_table(columns=2, rows=24, text=wheel_specs_tab, text_align='center')
    mdFile.new_line()
    
    mdFile.new_header(level=2, title="Computed specifications")
    mdFile.new_line()
    mdFile.write("""> **Consumption colouring** is based on the ratio between average speed and average consumption. It is considered good when below 0.75 and bad when above 1.0 """)
    mdFile.new_line()
    mdFile.new_line()
    mdFile.write("""> **Voltage sag colouring** is red when the coefficient is lower than -0.6 (strong voltage sag)""")
    mdFile.new_line()
    mdFile.new_table(columns=2, rows=4, text=computed_specs_tab, text_align='center')
    mdFile.new_line()
    mdFile.new_line()
    mdFile.write("> Based on "+str(len(wheel_trips))+" trips.")
    mdFile.new_line()
    mdFile.new_line()

    mdFile.new_header(level=1, title="Usage Statistcs")
    
    mdFile.new_header(level=2, title="Number of trips per source")
    mdFile.write(apps.to_html())
    
    mdFile.new_header(level=2, title="Number of trips per type")
    mdFile.write(trips.to_html())
    
    mdFile.new_header(level=2, title="Trips environment statistics")
    mdFile.write(env.to_html())
    
    mdFile.new_header(level=2, title="Trips energy statistics")
    mdFile.write(conso.to_html())
    
    mdFile.new_header(level=2, title="Trips statistics per field")
    
    for d in dfs:
        field=dfs.get(d)
        field=field.style.background_gradient()
        mdFile.write(field.to_html())
        mdFile.new_line()
        mdFile.write('---')
        mdFile.new_line()
        mdFile.new_line()
    
    mdFile.new_header(level=2, title="Trips details")
    trips_links=[]
    anal_files=os.listdir(analysis_dir)
    anal_files=[file for file in anal_files if file.endswith('.md')]
    anal_files.sort()
    for a in anal_files:
        trip_id=a.split('_')[-1]
        trip_id=trip_id.split('.')[0]
        trips_links+=['Trip '+trip_id,'[Details]'+'('+wheel+'/'+a+')']
    trips_links=['**Trip id**','**link**']+trips_links
    
    mdFile.new_table(columns=2, rows=len(anal_files)+1, text=trips_links, text_align='center')
    
    mdFile.create_md_file()

In [3]:
trips_root="trips/"
brands = os.listdir(trips_root)
brands = [ brand for brand in brands if not "ipynb" in brand]
for brand in brands:
    wheels = os.listdir(trips_root+brand)
    wheels = [ wheel for wheel in wheels if not "ipynb" in wheel]
    for wheel in wheels:
        if os.path.exists('analysis/'+wheel+'/'+wheel+'_0.md'):
            print("CREATING ",wheel,"PAGE")
            create_euc_page(wheel)

CREATING  lynx PAGE
analysis/lynx
CREATING  s22_pro PAGE
analysis/s22_pro


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python

CREATING  mten4 PAGE
analysis/mten4
CREATING  blitz PAGE
analysis/blitz


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



CREATING  master_pro_V2 PAGE
analysis/master_pro_V2
CREATING  v9 PAGE
analysis/v9


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered



CREATING  z10 PAGE
analysis/z10
CREATING  aero PAGE
analysis/aero


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python

CREATING  apex PAGE
analysis/apex


/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3807: RuntimeWarning:

All-NaN slice encountered

/home/tritbool/Workspace/.venv/lib/python3.13/site-packages/pandas/io/formats/style.py:3808: RuntimeWarning:

All-NaN slice encountered

